# Setup
Import libraries, etc.

In [1]:
! pip install evaluate bert_score rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=abb752f0d3b5f38bbcb57198799092491bc654addf767fcbfdff832ed7abb108
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM
import torch
from evaluate import load
from tqdm.notebook import tqdm
import numpy as np

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
rouge_score = load('rouge')
bert_score = load("bertscore")

# Dataset
Hasil scraping review dari Zomato

In [6]:
df = pd.read_csv('/kaggle/input/zomatoreview/zomato_reviews.csv')
df.head()

,resto_name,review_text,timestamp
0,Cé La Vie Kitchen & Bar,"They are offering false offers in Zomato gold,...",5 jam yang lalu
1,Cé La Vie Kitchen & Bar,The ambience and hygiene was just not it. The ...,15 jam yang lalu
2,Cé La Vie Kitchen & Bar,Very nice ambience with amazing vibesPrem kuma...,16 jam yang lalu
3,Cé La Vie Kitchen & Bar,They denied to take payment from Zomato gold ....,17 jam yang lalu
4,Cé La Vie Kitchen & Bar,It was good experience at sanam,19 jam yang lalu


# Eksperimen
Eksperimen akan menggunakan pendekatan LLM dengan task summarization dan generative. Untuk task summarization akan membandingkan kinerja summarization dari 3 buah model untuk pekerjaan summarization teks. Prompt yang digunakan adalah zero-shot.

Model yang digunakan antara lain adalah
1. facebook/bart-large-cnn
2. google-t5/t5-3b
3. google/bigbird-pegasus-large-arxiv

Untuk task generative, model yang digunakan adalah model-model sebagai berikut
1. Qwen/Qwen2.5-1.5B-Instruct.
2. Nexusflow/Athene-V2-Chat
3. google/gemma-2-2b-it

# Prompt using LLM Tuned for Text Summarization

In [7]:
base_prompt = "Summarize: "
print(base_prompt)

Summarize: 


## Bart-Large-CNN

In [8]:
bart_large_cnn_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_large_cnn_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [9]:
bart_large_cnn_result = {"input":[], "summary":[], "restaurant": []}

In [10]:
for resto in df['resto_name'].unique():
    text_inputs = [rev for rev in df[df['resto_name'] == resto]['review_text']]
    
    prompt = base_prompt
    for text in tqdm(text_inputs):
        prompt += text + "\n"
    
    inputs = bart_large_cnn_tokenizer(prompt, return_tensors='pt', truncation=True, padding=True, max_length=1024)
    
    with torch.no_grad():
        outputs = bart_large_cnn_model.generate(**inputs, max_new_tokens=100)
    
    full_summary = bart_large_cnn_tokenizer.decode(outputs[0], skip_special_tokens=True).strip() + " "
    
    bart_large_cnn_result['input'].append(prompt)
    bart_large_cnn_result['summary'].append(full_summary)
    bart_large_cnn_result['restaurant'].append(resto)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

In [11]:
bart_large_cnn_rougescore = rouge_score.compute(references=bart_large_cnn_result['input'], predictions=bart_large_cnn_result['summary'])
bart_large_cnn_rougescore

{'rouge1': 0.16385047132049152,
 'rouge2': 0.14659962665489618,
 'rougeL': 0.14863955594806727,
 'rougeLsum': 0.16215938331454594}

In [12]:
bart_large_cnn_bertscore = bert_score.compute(references=bart_large_cnn_result['input'], predictions=bart_large_cnn_result['summary'], model_type="distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [13]:
for k, v in bart_large_cnn_bertscore.items():
    if k=='hashcode':
        break
    average_score = np.average(v)
    print(k + ": " + str(average_score))

precision: 0.8931073461260114
recall: 0.7410896165030343
f1: 0.8083746944155011


In [14]:
bart_large_cnn_result_df = pd.DataFrame(bart_large_cnn_result)
bart_large_cnn_result_df.to_csv('bart_large_cnn_result.csv', index=None)

## T5 - large

In [15]:
t5_large_tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-large")
t5_large_model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-large")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [16]:
t5_large_result = {"input":[], "summary":[], "restaurant": []}

In [17]:
for resto in df['resto_name'].unique():
    text_inputs = [rev for rev in df[df['resto_name'] == resto]['review_text']]
    
    prompt = base_prompt
    for text in tqdm(text_inputs):
        prompt += text + "\n"
    
    inputs = t5_large_tokenizer(prompt, return_tensors='pt', truncation=True, padding=True, max_length=1024)
    
    with torch.no_grad():
        outputs = t5_large_model.generate(**inputs, max_new_tokens=100)
    
    full_summary = t5_large_tokenizer.decode(outputs[0], skip_special_tokens=True).strip() + " "
    
    t5_large_result['input'].append(prompt)
    t5_large_result['summary'].append(full_summary)
    t5_large_result['restaurant'].append(resto)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

In [18]:
t5_large_rougescore = rouge_score.compute(references=t5_large_result['input'], predictions=t5_large_result['summary'])
t5_large_rougescore

{'rouge1': 0.14270787210120753,
 'rouge2': 0.11611911599527772,
 'rougeL': 0.12581653672667364,
 'rougeLsum': 0.14080451363093477}

In [19]:
t5_large_bertscore = bert_score.compute(references=t5_large_result['input'], predictions=t5_large_result['summary'], model_type='distilbert-base-uncased')
t5_large_bertscore

{'precision': [0.8633115887641907,
  0.8255318403244019,
  0.8865184187889099,
  0.9001997709274292,
  0.8930476307868958,
  0.8191195726394653,
  0.8086378574371338],
 'recall': [0.7001564502716064,
  0.6727620363235474,
  0.7406982183456421,
  0.8827947378158569,
  0.7373613119125366,
  0.6705561876296997,
  0.6508636474609375],
 'f1': [0.7732210755348206,
  0.7413585186004639,
  0.8070746660232544,
  0.8914123177528381,
  0.8077712655067444,
  0.7374298572540283,
  0.721222996711731],
 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=4.46.3)'}

In [20]:
t5_large_result_df = pd.DataFrame(t5_large_result)
t5_large_result_df.to_csv('t5_large_result.csv', index=None)

## Pegasus

In [21]:
pegasus_tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")
pegasus_model = AutoModelForSeq2SeqLM.from_pretrained("google/bigbird-pegasus-large-arxiv")

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.92M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.31G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

In [22]:
pegasus_result = {"input":[], "summary":[], "restaurant": []}

In [23]:
for resto in df['resto_name'].unique():
    text_inputs = [rev for rev in df[df['resto_name'] == resto]['review_text']]
    
    prompt = base_prompt
    for text in tqdm(text_inputs):
        prompt += text + "\n"
    
    inputs = pegasus_tokenizer(prompt, return_tensors='pt', truncation=True, padding=True, max_length=1024)

    with torch.no_grad():
        outputs = pegasus_model.generate(**inputs, max_new_tokens=100)
    
    full_summary = pegasus_tokenizer.decode(outputs[0], skip_special_tokens=True).strip() + " "
    
    pegasus_result['input'].append(prompt)
    pegasus_result['summary'].append(full_summary)
    pegasus_result['restaurant'].append(resto)

  0%|          | 0/50 [00:00<?, ?it/s]

Input ids are automatically padded from 985 to 1024 to be a multiple of `config.block_size`: 64


model.safetensors:   0%|          | 0.00/2.31G [00:00<?, ?B/s]

  0%|          | 0/25 [00:00<?, ?it/s]

Input ids are automatically padded from 947 to 960 to be a multiple of `config.block_size`: 64


...

  0%|          | 0/10 [00:00<?, ?it/s]

Attention type 'block_sparse' is not possible if sequence_length: 234 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

In [24]:
pegasus_rougescore = rouge_score.compute(references=pegasus_result['input'], predictions=pegasus_result['summary'])
pegasus_rougescore

{'rouge1': 0.08983135313358061,
 'rouge2': 0.009865523260856232,
 'rougeL': 0.06619238812811107,
 'rougeLsum': 0.08209137315812093}

In [25]:
pegasus_bertscore = bert_score.compute(references=pegasus_result['input'], predictions=pegasus_result['summary'], model_type='distilbert-base-uncased')
pegasus_bertscore

{'precision': [0.706061840057373,
  0.7273514270782471,
  0.6786103248596191,
  0.6672173738479614,
  0.7368711829185486,
  0.7423299551010132,
  0.7025307416915894],
 'recall': [0.6516684889793396,
  0.6545687913894653,
  0.650047779083252,
  0.6160447597503662,
  0.679263710975647,
  0.6011269092559814,
  0.6324819326400757],
 'f1': [0.6777756214141846,
  0.6890434622764587,
  0.6640220880508423,
  0.6406107544898987,
  0.7068957090377808,
  0.6643079519271851,
  0.6656686067581177],
 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=4.46.3)'}

In [27]:
pegasus_result_df = pd.DataFrame(pegasus_result)
pegasus_result_df.to_csv('pegasus_result.csv', index=None)

## Model yang Dipilih
Model yang digunakan adalah T5 - large karena kemampuan yang dapat digunakan untuk melakukan summary secara abstractive.

In [28]:
df_sentiment = pd.read_csv('/kaggle/input/zomatoreview/deberta-v3-base-absa-v1.csv')
df_sentiment.head()

,Unnamed: 0,text,category,topic_food,score_food,topic_place,score_place,topic_price,score_price,topic_service,score_service,topic_count,main_topics,primary_topic,primary_score,sentiment_food,sentiment_place,sentiment_price,sentiment_service
0,0,"To be completely fair, the only redeeming fact...",food,1,0.223695,0,0.000000,0,0.0,0,0.0,1,food,food,0.223695,Positive,Not Found,Not Found,Not Found
1,1,"The food is uniformly exceptional, with a very...",food,1,0.515413,1,0.378481,0,0.0,0,0.0,2,"food, place",food,0.515413,Positive,Neutral,Not Found,Not Found
2,2,"Not only was the food outstanding, but the lit...",food,1,0.330244,0,0.000000,0,0.0,0,0.0,1,food,food,0.330244,Positive,Not Found,Not Found,Not Found
3,3,It is very overpriced and not very tasty.,food,1,0.371381,0,0.000000,0,0.0,0,0.0,1,food,food,0.371381,Negative,Not Found,Not Found,Not Found
4,4,Our agreed favorite is the orrechiete with sau...,food,1,0.424373,1,0.321227,0,0.0,0,0.0,2,"food, place",food,0.424373,Neutral,Neutral,Not Found,Not Found


In [29]:
result = {"category":[], "sentiment":[], "summary":[]}

In [ ]:
for category in df_sentiment['category'].unique():
    # Get each sentiment for the category
    print("Category " + category)
    for sentiment in ['Positive', 'Negative', 'Neutral']:
        print("Sentiment " + sentiment)
        prompt = "Summarize: "
        
        for text in tqdm(df_sentiment[df_sentiment['sentiment_' + category] == sentiment]['text']):
            prompt += text + "\n"
            
        # text_chunks = [prompt[i:i + 512] for i in range(0, len(prompt), 512)]
        full_summary = ""
        
        # for chunk in text_chunks:
        inputs = t5_large_tokenizer(prompt, return_tensors='pt')
        
        with torch.no_grad():
            outputs = t5_large_model.generate(**inputs, max_new_tokens=100)
        
        full_summary = t5_large_tokenizer.decode(outputs[0], skip_special_tokens=True).strip() + " "
        print(full_summary)

        result['category'].append(category)
        result['sentiment'].append(sentiment)
        result['summary'].append(full_summary)

In [ ]:
final_df = pd.DataFrame(result, index=None)
final_df.to_csv('result.csv')

# Prompt using LLM for Text Generation

## Prompt

In [31]:
base_prompt = "Summarize the following reviews to give out the aspects that it talks most about:\n"
print(base_prompt)

Summarize the following reviews to give out the aspects that it talks most about:



## Qwen/Qwen2.5-1.5B-Instruct

In [32]:
qwen_result = {'input':[], 'summary': [], 'restaurant': []}

In [33]:
qwen_model = AutoModelForCausalLM.from_pretrained('Qwen/Qwen2.5-1.5B-Instruct', device_map='auto')
qwen_tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2.5-1.5B-Instruct')

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [34]:
for resto in df['resto_name'].unique():
    text_inputs = [rev for rev in df[df['resto_name'] == resto]['review_text']]
    
    prompt = base_prompt
    for text in tqdm(text_inputs):
        prompt += text + "\n"
    
    inputs = qwen_tokenizer(prompt, return_tensors='pt').to(device)
    
    with torch.no_grad():
        outputs = qwen_model.generate(**inputs, max_new_tokens=100)
    
    full_summary = qwen_tokenizer.decode(outputs[:, inputs['input_ids'].shape[-1]:][0], 
                                         skip_special_tokens=True).strip() + " "
    
    qwen_result['input'].append(prompt)
    qwen_result['summary'].append(full_summary)
    qwen_result['restaurant'].append(resto)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

In [35]:
qwen_rougescore = rouge_score.compute(references=qwen_result['input'], predictions=qwen_result['summary'])
qwen_rougescore

{'rouge1': 0.15774801508174954,
 'rouge2': 0.08894305300797486,
 'rougeL': 0.11839630708574284,
 'rougeLsum': 0.1481931878008779}

In [36]:
qwen_bertscore = bert_score.compute(references=qwen_result['input'], predictions=qwen_result['summary'], model_type='distilbert-base-uncased')
qwen_bertscore

{'precision': [0.8029330968856812,
  0.8279682993888855,
  0.7903814911842346,
  0.8016378879547119,
  0.8018168807029724,
  0.772672176361084,
  0.7934010028839111],
 'recall': [0.7292390465736389,
  0.7114908695220947,
  0.7286142110824585,
  0.8401410579681396,
  0.725367546081543,
  0.6935974359512329,
  0.7035403251647949],
 'f1': [0.7643137574195862,
  0.7653231620788574,
  0.7582420110702515,
  0.8204379677772522,
  0.7616786956787109,
  0.7310025691986084,
  0.7457734942436218],
 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=4.46.3)'}

In [37]:
qwen_result_df = pd.DataFrame(qwen_result)
qwen_result_df.to_csv('qwen_result.csv', index=None)

## Qwen/Qwen2.5-0.5B

In [ ]:
small_qwen_result = {'input':[], 'summary': [], 'restaurant': []}

In [ ]:
small_qwen_model = AutoModelForCausalLM.from_pretrained('Qwen/Qwen2.5-0.5B', device_map='auto', trust_remote_code=True)
small_qwen_tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2.5-0.5B', trust_remote_code=True)

In [ ]:
for resto in df['resto_name'].unique():
    text_inputs = [rev for rev in df[df['resto_name'] == resto]['review_text']]
    
    prompt = base_prompt
    for text in tqdm(text_inputs):
        prompt += text + "\n"
    
    inputs = small_qwen_tokenizer(prompt, return_tensors='pt').to(device)
    
    with torch.no_grad():
        outputs = small_qwen_model.generate(**inputs, max_new_tokens=100)
    
    full_summary = small_qwen_tokenizer.decode(outputs[:, inputs['input_ids'].shape[-1]:][0], 
                                         skip_special_tokens=True).strip() + " "
    
    small_qwen_result['input'].append(prompt)
    small_qwen_result['summary'].append(full_summary)
    small_qwen_result['restaurant'].append(resto)

In [ ]:
small_qwen_rougescore = rouge_score.compute(references=small_qwen_result['input'], predictions=small_qwen_result['summary'])
small_qwen_rougescore

In [ ]:
small_qwen_bertscore = bert_score.compute(references=small_qwen_result['input'], predictions=small_qwen_result['summary'], model_type='distilbert-base-uncased')
small_qwen_bertscore

In [ ]:
small_qwen_result_df = pd.DataFrame(small_qwen_result)
small_qwen_result_df.to_csv('small_qwen_result.csv', index=None)

## numind/NuExtract-1.5

In [ ]:
numind_result = {'input':[], 'summary': [], 'restaurant': []}

In [ ]:
numind_model = AutoModelForCausalLM.from_pretrained('numind/NuExtract-1.5', device_map='auto', trust_remote_code=True)
numind_tokenizer = AutoTokenizer.from_pretrained('numind/NuExtract-1.5', trust_remote_code=True)

In [ ]:
for resto in df['resto_name'].unique():
    text_inputs = [rev for rev in df[df['resto_name'] == resto]['review_text']]
    
    prompt = base_prompt
    for text in tqdm(text_inputs):
        prompt += text + "\n"
    
    inputs = numind_tokenizer(prompt, return_tensors='pt').to(device)
    
    with torch.no_grad():
        outputs = numind_model.generate(**inputs, max_new_tokens=100)
    
    full_summary = numind_tokenizer.decode(outputs[:, inputs['input_ids'].shape[-1]:][0], 
                                         skip_special_tokens=True).strip() + " "
    
    numind_result['input'].append(prompt)
    numind_result['summary'].append(full_summary)
    numind_result['restaurant'].append(resto)

In [ ]:
numind_rougescore = rouge_score.compute(references=numind_result['input'], predictions=numind_result['summary'])
numind_rougescore

In [ ]:
numind_bertscore = bert_score.compute(references=numind_result['input'], predictions=numind_result['summary'], model_type='distilbert-base-uncased')
numind_bertscore

In [ ]:
numind_result_df = pd.DataFrame(numind_result)
numind_result_df.to_csv('numind_result.csv', index=None)

## Model yang Dipilih

Model yang dipilih adalah model QWEN2.5-1.5B-Instruct

In [38]:
result_gen = {'summary':[], 'category':[], 'sentiment':[]}

In [39]:
for category in df_sentiment['category'].unique():
    # Get each sentiment for the category
    print("Category " + category)
    for sentiment in ['Positive', 'Negative', 'Neutral']:
        print("Sentiment " + sentiment)
        prompt = "Summarize the following reviews to give out the aspects that it talks most about:\n"
        
        for text in tqdm(df_sentiment[df_sentiment['sentiment_' + category] == sentiment]['text']):
            prompt += text + "\n"
            
        full_summary = ""
        
        inputs = qwen_tokenizer(prompt, return_tensors='pt').to(device)
        
        with torch.no_grad():
            outputs = qwen_model.generate(**inputs, max_new_tokens=100)
        
        full_summary = qwen_tokenizer.decode(outputs[:, inputs['input_ids'].shape[-1]:][0], skip_special_tokens=True).strip() + " "
        print(full_summary)

        result_gen['category'].append(category)
        result_gen['sentiment'].append(sentiment)
        result_gen['summary'].append(full_summary)

Category food
Sentiment Positive


  0%|          | 0/515 [00:00<?, ?it/s]

The waiters are sweet, the food is tasty and the bill is never too large.
Service here was great, food was fantastic.
The service is excellent, the decor is great, and the food is delicious and comes in large portions.
The place is small and cramped but the food is fantastic.
The food is yummy, especially their cooked-to-perfection mussels in spicy tomato sauce and their shoestring crispy fries.
The food is inventive but still keeps traditional indian flavoring.
The waiters are 
Sentiment Negative


  0%|          | 0/137 [00:00<?, ?it/s]

I have been a longtime fan of Holy Basil in the East Village, and while I do believe their food has slightly slipped in quality, I have been hesitant to be disloyal.
The food can get pricey but the prixe fixe tasting menu is the greatest food for a good price and they cater the food to any food allergies or food you don't like.
The food however, is what one might expect.
I ate here a week ago and found most dishes average at best and too expensive 
Sentiment Neutral


...

I thought it was a very nice place and the food was pretty good.
The service was okay, but the food was pretty good.
The food is fantastic, the service is top notch, and the atmosphere is great.
The place is a great choice for a nice, casual dinner with friends or a date.
The food was great, the service was great, and the atmosphere was great.
We decided to go for lunch and ended up staying until 3 o'clock, leaving our dog home alone. 
Sentiment Negative


  0%|          | 0/145 [00:00<?, ?it/s]

It is a small restuarant so go early and it will be nice and quiet- go late and it will be packed!
The interior is bright and airy and it feels like a home away from home.
The decor is really blah, and not at all hip or happening.
The atmosphere is too loud, but I think that was because of our party!
The lighting is dimmed and the atmosphere is cozy.
You have to go through a couple of doors to get to the main dining room, 
Sentiment Neutral


  0%|          | 0/451 [00:00<?, ?it/s]

The food is awesome, it's light and easy to digest, and the presentation is exquisite.
The food is average, and I would say even the chain restaurant Baluchi's tastes better.
You will pay a lot for the decore, but the food is no better or worse than a lot of other Chinese and Asian fusion places in NY.
I care more about the food and ambience.
The food is average, and I would say even the chain restaurant Baluchi's tastes better.
The service 


In [40]:
result_gen_df = pd.DataFrame(result_gen)
result_gen_df.to_csv('result_gen_result.csv', index=None)